In [1]:
import pandas as pd
from pathlib import Path
import re
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import altair as alt
from vega_datasets import data



BASE_DIR = Path("..")
ORIGINAL_DATA_DIR = BASE_DIR / "original_data"
CLEAN_DATA_DIR = BASE_DIR / "clean_data"
FILE_PATH_TERM = CLEAN_DATA_DIR / "nsf_terminations_airtable.csv"
FILE_PATH_UNI = CLEAN_DATA_DIR / "heis_usa.csv"

In [2]:
terminations = pd.read_csv(FILE_PATH_TERM)

In [3]:
import altair as alt
import pandas as pd

# Comptar quantes terminacions té cada institució
grants_per_org = (
    terminations.groupby('org_name')
    .size()
    .reset_index(name='num_grants')
)

# Seleccionar el top 10
top10 = grants_per_org.nlargest(10, 'num_grants')
top10_names = top10['org_name'].tolist()

# Calcular les mitjanes
mean_all = grants_per_org['num_grants'].mean()
other_grants_per_org = grants_per_org[~grants_per_org['org_name'].isin(top10_names)]
mean_rest = other_grants_per_org['num_grants'].mean() if not other_grants_per_org.empty else float('nan')

# Límite superior fix de l'eix Y
ymax = 310

# --- Gràfic de barres verticals ---
bars = alt.Chart(top10).mark_bar(color='lightsteelblue').encode(
    x=alt.X('org_name:N', sort='-y', title='Institució'),
    y=alt.Y('num_grants:Q',
            title='Nombre de grants acabades',
            scale=alt.Scale(domain=[0, ymax], nice=False)),  # <-- scale fix amb nice=False
    tooltip=['org_name', 'num_grants']
).properties(
    title='Top 10 institucions amb més grants acabades'
)

# --- Dades de les mitjanes ---
mean_data = pd.DataFrame({
    'y': [mean_all, mean_rest],
    'year': ['Mitjana total', 'Mitjana resta institucions']
})

# --- Línies de mitjana (contínues i més gruixudes) ---
mean_lines = alt.Chart(mean_data).mark_rule(strokeWidth=3).encode(
    y='y:Q',
    color=alt.Color(
        'year:N',
        scale=alt.Scale(
            domain=['Mitjana total', 'Mitjana resta institucions'],
            range=['darkred', 'darkorange']
        ),
        title='Línies de mitjana'
    )
)

# --- Gràfic final combinat ---
chart = (bars + mean_lines).properties(
    width=700,
    height=420
)

chart


alt.LayerChart(...)

In [4]:
import altair as alt
import pandas as pd

# Comptar quantitats de grants per institució
grants_per_org = terminations.groupby('org_name').size().reset_index(name='num_grants')
budget_per_org = terminations.groupby('org_name')['nsf_total_budget'].sum().reset_index()

# Top institucions
top10_grants = grants_per_org.nlargest(10, 'num_grants')
top10_names = top10_grants['org_name'].tolist()
top10_budget = budget_per_org.nlargest(10, 'nsf_total_budget')
top10_budget_names = top10_budget['org_name'].tolist()
all_institutions = list(set(top10_names + top10_budget_names))

# Dades combinades
plot_data = grants_per_org.merge(budget_per_org, on='org_name')
plot_data = plot_data[plot_data['org_name'].isin(all_institutions)].sort_values('num_grants', ascending=False)

# Afegir URL del logo i posicions x equitatives
plot_data['logo_url'] = 'https://upload.wikimedia.org/wikipedia/commons/thumb/2/2f/University_of_California_logo.svg/800px-University_of_California_logo.svg.png'
plot_data['x_position'] = range(len(plot_data))

# Mitjanes
mean_all_grants = grants_per_org['num_grants'].mean()
mean_rest_grants = grants_per_org.loc[~grants_per_org['org_name'].isin(top10_names), 'num_grants'].mean()
mean_all_budget = budget_per_org['nsf_total_budget'].mean()
mean_rest_budget = budget_per_org.loc[~budget_per_org['org_name'].isin(top10_budget_names), 'nsf_total_budget'].mean()

# Escales màximes
ymax_grants = plot_data['num_grants'].max() * 1.2
ymax_budget = plot_data['nsf_total_budget'].max() * 1.2

# --- BARRES (nombre de grants) ---
bars = alt.Chart(plot_data).mark_bar(color='#bcdfff').encode(
    x=alt.X('x_position:Q', axis=alt.Axis(labels=False, ticks=False, title=None)),
    y=alt.Y('num_grants:Q', title='Nombre de grants', scale=alt.Scale(domain=[0, ymax_grants])),
    tooltip=['org_name', 'num_grants', 'nsf_total_budget']
)

# --- PUNTS (diners, eix dret) ---
points = alt.Chart(plot_data).mark_point(filled=True, size=150, color='#2ca02c').encode(
    x=alt.X('x_position:Q'),
    y=alt.Y('nsf_total_budget:Q',
            title='Diners cancel·lats ($)',
            scale=alt.Scale(domain=[0, ymax_budget])),
    tooltip=['org_name', 'nsf_total_budget']
)

# --- LÍNIES DE MITJANES ---
mean_data = pd.DataFrame({
    'label': [
        'Mitjana total grants', 'Mitjana resta grants',
        'Mitjana total diners', 'Mitjana resta diners'
    ],
    'value': [mean_all_grants, mean_rest_grants, mean_all_budget, mean_rest_budget],
    'type': ['grants', 'grants', 'budget', 'budget']
})

color_scale = alt.Scale(
    domain=[
        'Mitjana total grants',
        'Mitjana resta grants',
        'Mitjana total diners',
        'Mitjana resta diners'
    ],
    range=['#1f77b4', '#ff7f0e', '#2ca02c', '#9467bd']
)

# Línies de mitjana per a grants (eix esquerre)
mean_lines_grants = alt.Chart(mean_data.query("type == 'grants'")).mark_rule(strokeDash=[5, 3]).encode(
    y=alt.Y('value:Q', title='Nombre de grants'),
    color=alt.Color('label:N', scale=color_scale, title='Mitjanes')
)

# Línies de mitjana per a diners (eix dret)
mean_lines_budget = alt.Chart(mean_data.query("type == 'budget'")).mark_rule(strokeDash=[5, 3]).encode(
    y=alt.Y('value:Q',
            axis=alt.Axis(title='Diners cancel·lats ($)'),
            scale=alt.Scale(domain=[0, ymax_budget])),
    color=alt.Color('label:N', scale=color_scale, title='Mitjanes')
)

# --- GRÀFIC PRINCIPAL ---
main_chart = alt.layer(
    bars + mean_lines_grants,
    points + mean_lines_budget
).resolve_scale(
    y='independent'
).properties(
    width=850,
    height=450,
    title='Top institucions: nombre de grants i diners cancel·lats (ordenat per nombre de grants)'
)

# --- LOGOS (en un gràfic separat sense eix Y) ---
logos = alt.Chart(plot_data).mark_image(
    width=60,
    height=60
).encode(
    x=alt.X('x_position:Q', axis=None, scale=alt.Scale(domain=[-0.5, len(plot_data)-0.5])),
    url='logo_url:N'
).properties(
    width=850,
    height=60
)

# --- TEXT "INSTITUCIÓ" ---
institution_label = alt.Chart(pd.DataFrame({'label': ['Institució']})).mark_text(
    align='center',
    baseline='top',
    fontSize=12,
).encode(
    text='label:N'
).properties(
    width=850,
    height=20
)

# --- COMBINAR VERTICALMENT ---
chart = alt.vconcat(
    main_chart,
    logos,
    institution_label,
    spacing=2
).resolve_scale(
    x='shared'
).configure_view(
    strokeWidth=0
)

chart

alt.VConcatChart(...)